In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
def load_corr(modalities, table_name):
    df_att_list = []
    df_unatt_list = []
    for modality in modalities:
        df = pd.read_csv(f'tables/Overlay/{modality}/{table_name}')
        cc1_cc2 = df[df['Component'].isin(['CC 1', 'CC 2'])].copy()
        cc1_cc2['Modality'] = modality
        df_att_list.append(cc1_cc2[['Subject ID', 'Component', 'Att', 'Modality']])
        df_unatt_list.append(cc1_cc2[['Subject ID', 'Component', 'Unatt', 'Modality']])
    corr_test_att = pd.concat(df_att_list, ignore_index=True)
    corr_test_unatt = pd.concat(df_unatt_list, ignore_index=True)
    return corr_test_att, corr_test_unatt

def load_acc(modalities, table_name):
    acc = []
    for modality in modalities:
        table = pd.read_csv('tables/Overlay/' + modality + '/' + table_name)
        table['Modality'] = modality
        table = table.rename(columns={'Trial_len=30': 'Accuracy'})
        acc.append(table[['Subject ID', 'Accuracy', 'Modality']])
    acc_df = pd.concat(acc)
    return acc_df

def plot_corr(df, metric, title, ax):
    df_long = pd.melt(df, id_vars=['Subject ID', 'Modality', 'Component'], var_name='Metric', value_name='Value')
    df_long = df_long[df_long['Metric'] == metric]
    sns.boxplot(x='Modality', y='Value', hue='Component', data=df_long, showfliers=False, dodge=True, ax=ax)
    sns.stripplot(x='Modality', y='Value', hue='Component', data=df_long, jitter=True, dodge=True, alpha=0.5, edgecolor='gray', linewidth=0.5, ax=ax)
    ax.set_title(title)
    ax.set_ylabel('Correlation Coefficient')
    ax.set_xlabel('Modality')
    ax.legend()

def plot_acc(df, title, ax):
    sns.boxplot(x='Modality', y='Accuracy', data=df, color='lightgray', ax=ax)
    sns.stripplot(x='Modality', y='Accuracy', hue='Subject ID', data=df, jitter=True, ax=ax, legend=False)
    ax.set_title(title)
    ax.set_ylabel('Accuracy')
    ax.set_xlabel('Modality')
    ax.legend()

def adjust_legend(axes):
    for ax in axes:
        handles, labels = ax.get_legend_handles_labels()
        unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
        ax.legend(*zip(*unique))

In [ ]:
modalities = ['EEG', 'EOG', 'GAZE_V', 'EOG_V', 'SACC']
table_name = 'ObjFlow_Corr_Train_Att_Mask_False.csv'
corr_test_att, corr_test_unatt = load_corr(modalities, table_name)

plt.close('all')
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

plot_corr(corr_test_att, 'Att', 'Train on Attended, Test on Attended', axes[0])
plot_corr(corr_test_unatt, 'Unatt', 'Train on Attended, Test on Unattended', axes[1])
adjust_legend(axes)

plt.tight_layout()
plt.show()

In [ ]:
modalities = ['EEG', 'EOG', 'GAZE_V', 'EOG_V', 'SACC']
table_name = 'ObjFlow_Corr_Train_Unatt_Mask_False.csv'
corr_test_att, corr_test_unatt = load_corr(modalities, table_name)

plt.close('all')
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

plot_corr(corr_test_att, 'Att', 'Train on Unattended, Test on Attended', axes[0])
plot_corr(corr_test_unatt, 'Unatt', 'Train on Unattended, Test on Unattended', axes[1])
adjust_legend(axes)

plt.tight_layout()
plt.show()

In [ ]:
table_name = 'ObjFlow_Corr_Train_Att_Mask_False.csv'
modalities = ['EEG', 'EOG', 'GAZE_V']
corr_test_ori_att, corr_test_ori_unatt = load_corr(modalities, table_name)
modalities_reg = ['EEG-EOG', 'EEG-GAZE_V', 'GAZE_V-EEG', 'EEG-EOG&GAZE_V']
corr_test_reg_att, corr_test_reg_unatt = load_corr(modalities_reg, table_name)

In [ ]:
plt.close('all')
fig, axes = plt.subplots(1, 2, figsize=(11, 4), sharey=True)

plot_corr(corr_test_ori_att, 'Att', 'Train on Attended, Test on Attended', axes[0])
plot_corr(corr_test_reg_att, 'Att', 'Train on Attended, Test on Attended', axes[1])
adjust_legend(axes)

plt.tight_layout()
plt.show()

In [ ]:
plt.close('all')
fig, axes = plt.subplots(1, 2, figsize=(11, 4), sharey=True)

plot_corr(corr_test_ori_unatt, 'Unatt', 'Train on Attended, Test on Unattended', axes[0])
plot_corr(corr_test_reg_unatt, 'Unatt', 'Train on Attended, Test on Unattended', axes[1])
adjust_legend(axes)

plt.tight_layout()
plt.show()

In [ ]:
modalities = ['EEG', 'EOG', 'GAZE_V', 'EOG_V', 'SACC']
table_name = 'ObjFlow_Acc_Train_Att_Mask_False.csv'
acc_mask_false_df = load_acc(modalities, table_name)

modalities = ['EEG+EOG', 'EEG+GAZE_V', 'EOG+GAZE_V', 'EEG+EOG+GAZE_V']
acc_mask_false_cb_df = load_acc(modalities, table_name)

modalities = ['EEG-EOG', 'EEG-GAZE_V', 'GAZE_V-EEG', 'EEG-EOG&GAZE_V']
acc_mask_false_reg_df = load_acc(modalities, table_name)

modalities = ['EEG', 'EOG', 'GAZE_V', 'EOG_V', 'SACC']
table_name = 'ObjFlow_Acc_MM_Train_Att_Mask_False.csv'
acc_mm_att_mask_false_df = load_acc(modalities, table_name)

table_name = 'ObjFlow_Acc_MM_Train_Unatt_Mask_False.csv'
acc_mm_unatt_mask_false_df = load_acc(modalities, table_name)

modalities = ['EEG', 'EOG', 'GAZE_V', 'EOG_V']
table_name = 'ObjFlow_Acc_Train_Att_Mask_True.csv'
acc_mask_true_df = load_acc(modalities, table_name)

In [ ]:
plt.close('all')
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

plot_acc(acc_mask_false_df, 'Compete-Att, Trial length=30s', axes[0])
plot_acc(acc_mm_att_mask_false_df, 'MM-Att, Trial length=30s', axes[1])
adjust_legend(axes)

plt.tight_layout()
plt.show()

In [ ]:

plt.close('all')
fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

plot_acc(acc_mask_false_df, 'Compete-Att', axes[0])
plot_acc(acc_mask_true_df, 'Compete-Att, Saccades Removed', axes[1])
adjust_legend(axes)

plt.tight_layout()
plt.show()

In [ ]:
modalities = ['EEG', 'EOG', 'GAZE_V']
table_name = 'ObjFlow_Acc_Train_Att_Mask_False.csv'
acc_mask_false_df_part = load_acc(modalities, table_name)

plt.close('all')
fig, axes = plt.subplots(1, 2, figsize=(11, 4), sharey=True)

plot_acc(acc_mask_false_df_part, 'Compete-Att', axes[0])
plot_acc(acc_mask_false_cb_df, 'Compete-Att, Multi-Modal', axes[1])
adjust_legend(axes)

plt.tight_layout()
plt.show()

In [ ]:
plt.close('all')
fig, axes = plt.subplots(1, 2, figsize=(11, 4), sharey=True)

plot_acc(acc_mask_false_df_part, 'Compete-Att', axes[0])
plot_acc(acc_mask_false_reg_df, 'Compete-Att, With Regression', axes[1])
adjust_legend(axes)

plt.tight_layout()
plt.show()

In [ ]:
plt.close('all')

fig, ax = plt.subplots(figsize=(5, 4))

plot_acc(acc_mm_unatt_mask_false_df, 'MM-Unatt, Trial length=30s', ax)

plt.tight_layout()
plt.show()